<a href="https://colab.research.google.com/github/BRkavin/AL-ML-/blob/main/lung_cancer_ANN_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer


In [12]:
# Sigmoid activation function and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


In [13]:
def sigmoid_derivative(x):
    return x * (1 - x)

In [14]:
# Mean squared error loss and its derivative
def mse_loss(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

In [15]:
def mse_loss_derivative(y_true, y_pred):
    return -2 * (y_true - y_pred)


In [16]:
class NeuralNetwork:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.weights_hidden = np.random.randn(input_dim, hidden_dim)
        self.bias_hidden = np.zeros((1, hidden_dim))
        self.weights_output = np.random.randn(hidden_dim, output_dim)
        self.bias_output = np.zeros((1, output_dim))

    def forward(self, X):
        self.hidden_layer_input = np.dot(X, self.weights_hidden) + self.bias_hidden
        self.hidden_layer_output = sigmoid(self.hidden_layer_input)
        self.output_layer_input = np.dot(self.hidden_layer_output, self.weights_output) + self.bias_output
        self.output_layer_output = sigmoid(self.output_layer_input)
        return self.output_layer_output

    def backward(self, X, y_true, y_pred, learning_rate):
        output_error = mse_loss_derivative(y_true, y_pred) * sigmoid_derivative(y_pred)
        hidden_error = np.dot(output_error, self.weights_output.T) * sigmoid_derivative(self.hidden_layer_output)

        delta_output = np.dot(self.hidden_layer_output.T, output_error)
        delta_hidden = np.dot(X.T, hidden_error)

        self.weights_output -= learning_rate * delta_output
        self.bias_output -= learning_rate * np.sum(output_error, axis=0, keepdims=True)
        self.weights_hidden -= learning_rate * delta_hidden
        self.bias_hidden -= learning_rate * np.sum(hidden_error, axis=0, keepdims=True)

    def train(self, X, y_true, epochs, learning_rate):
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = mse_loss(y_true, y_pred)
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss}")
            self.backward(X, y_true, y_pred, learning_rate)

    def predict(self, X):
        return np.round(self.forward(X))

In [17]:
# Load and preprocess data
data = load_breast_cancer()
X = data.data
y = data.target.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
# Initialize and train the neural network
input_dim = X_train.shape[1]
hidden_dim = 5
output_dim = 1
epochs = 1000
learning_rate = 0.01


In [19]:
nn = NeuralNetwork(input_dim, hidden_dim, output_dim)
nn.train(X_train, y_train, epochs, learning_rate)

Epoch 0, Loss: 0.21551686685815663
Epoch 10, Loss: 0.05597552418317584
Epoch 20, Loss: 0.04043105935143633
Epoch 30, Loss: 0.031107132420055358
Epoch 40, Loss: 0.024607816875275128
Epoch 50, Loss: 0.02170132631360665
Epoch 60, Loss: 0.019878755267647544
Epoch 70, Loss: 0.018608468122670304
Epoch 80, Loss: 0.01765400326717075
Epoch 90, Loss: 0.016901737373887917
Epoch 100, Loss: 0.016283344371025295
Epoch 110, Loss: 0.015756946071467527
Epoch 120, Loss: 0.015296188818932306
Epoch 130, Loss: 0.01488385586847296
Epoch 140, Loss: 0.014508285639783848
Epoch 150, Loss: 0.014161291998425569
Epoch 160, Loss: 0.013836896808269128
Epoch 170, Loss: 0.013530521875600958
Epoch 180, Loss: 0.013238469762073532
Epoch 190, Loss: 0.012957666054011469
Epoch 200, Loss: 0.012685777404528269
Epoch 210, Loss: 0.01242176153616876
Epoch 220, Loss: 0.012166197622218138
Epoch 230, Loss: 0.011920248632285137
Epoch 240, Loss: 0.011684022031176631
Epoch 250, Loss: 0.011456381786339438
Epoch 260, Loss: 0.01123591087

In [20]:
# Evaluate the model
y_pred = nn.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 97.37%
